In [1]:
import collections
import numpy as np
import pandas as pd
import scipy.sparse as sps
from datetime import datetime

### Preprocessing

Read the data.

In [2]:
col_names = ['user_id', 'friend_id']
df = pd.read_csv('Gowalla_edges.txt', sep='\t', names=col_names)

In [3]:
df.head()

,user_id,friend_id
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1900654 entries, 0 to 1900653
Data columns (total 2 columns):
user_id      int64
friend_id    int64
dtypes: int64(2)
memory usage: 29.0 MB


Read check-in information.

In [5]:
col_names = ['user_id', 'time', 'latitude', 'longitude', 'location_id']
df_checkins = pd.read_csv('Gowalla_totalCheckins.txt', sep='\t', names=col_names)

In [6]:
df_checkins.head()

,user_id,time,latitude,longitude,location_id
0,0,2010-10-19T23:55:27Z,30.235909,-97.795140,22847
1,0,2010-10-18T22:17:43Z,30.269103,-97.749395,420315
2,0,2010-10-17T23:42:03Z,30.255731,-97.763386,316637
3,0,2010-10-17T19:26:05Z,30.263418,-97.757597,16516
4,0,2010-10-16T18:50:42Z,30.274292,-97.740523,5535878


In [7]:
df_checkins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6442892 entries, 0 to 6442891
Data columns (total 5 columns):
user_id        int64
time           object
latitude       float64
longitude      float64
location_id    int64
dtypes: float64(2), int64(2), object(1)
memory usage: 245.8+ MB


In [8]:
df_checkins.drop(columns=['time', 'latitude', 'longitude'], inplace=True)

In [9]:
df_checkins['user_id'].nunique() == df['user_id'].nunique()

False

Remove users from the main dataframe which are not available in check-in data and renumber users.

All users need to offer recommendations, but this task assumes that the quality assessment is carried out on the basis of only those users who have check-ins. So, as for me, this action is justified.

In [10]:
absent = set(range(df['user_id'].nunique())) - set(df_checkins['user_id'].unique())

In [11]:
df = df[~df['user_id'].isin(absent) & ~df['friend_id'].isin(absent)]
df.reset_index(drop=True, inplace=True)

In [12]:
uid_to_idx, idx_to_uid = {}, {}
for idx, user_id in enumerate(df['user_id'].unique()):
    uid_to_idx[user_id] = idx
    idx_to_uid[idx] = user_id

In [13]:
def map_ids(row, mapper):
    return mapper[row]

In [14]:
df.loc[:, 'user_id'] = df['user_id'].apply(map_ids, args=[uid_to_idx]).values
df.loc[:, 'friend_id'] = df['friend_id'].apply(map_ids, args=[uid_to_idx]).values

Initialize S (Similarity), A (Availability) and R (Responsibility) matrices.

In [15]:
n_users = df['user_id'].nunique()

In [16]:
rows = df['user_id'].values
cols = df['friend_id'].values
data = np.ones(df.shape[0])

In [17]:
coo_mtx = sps.coo_matrix((data, (rows, cols)), shape=(n_users, n_users))

In [18]:
S = coo_mtx.tocsr()
S.setdiag(-1)

/usr/local/lib/python3.6/dist-packages/scipy/sparse/compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [19]:
A, R = S.copy(), S.copy()
A.data, R.data = np.zeros(len(S.data), dtype=np.float64), np.zeros(len(S.data), dtype=np.float64)

Some auxiliary arrays.

In [20]:
indptr = S.indptr
indices = S.indices
ind_range = np.arange(n_users)

In [21]:
nz_cols = [indices[start_idx:end_idx].tolist() 
           for start_idx, end_idx in zip(indptr, indptr[1:])]

In [22]:
idx_to_col = [dict(zip(ind_range, chunk)) for chunk in nz_cols]

### AP-algorithm implementation

In [23]:
def row_generator(X):
    for start_idx, end_idx in zip(indptr, indptr[1:]):
        yield X.data[start_idx:end_idx]

In [24]:
def row_max(X):
    m_ind, m_val = [], []

    for row in row_generator(X):
        m_ind.append(np.argmax(row))
        m_val.append(np.max(row))

    # assign the corresponding max value for each element of sparse matrix data
    m_arr = np.repeat(np.array(m_val).T, np.diff(indptr))
    return m_ind, m_val, m_arr

In [25]:
def update_r(A, S, R, damping=0.5):
    temp = S.copy()
    temp.data += A.data

    fm_ind, fm_val, fm_arr = row_max(temp)
    temp[ind_range, fm_ind] = -np.inf
    _, sm_val, sm_arr = row_max(temp)

    # replace first maximum values by second ones where col_idx is the index of max value
    for idx, val in enumerate(indptr[:-1]):
        col_idx = val + fm_ind[idx]
        fm_arr[col_idx] = sm_arr[col_idx]

    R_new = S.copy()
    R_new.data = R.data * damping + (1 - damping) * (R_new.data - fm_arr)
    return R_new

In [26]:
def update_a(R, A, damping=0.5):
    A_new = R.copy()
    A_new.setdiag(0)
    A_new[A_new < 0] = 0

    col_sums = np.sum(A_new, axis=0).A.flatten()
    arr_sums = R.diagonal() + col_sums
    
    # also substract values that should not be included into the sums
    A_new.data = np.minimum(0, arr_sums[indices] - A_new.data)

    A_new.setdiag(col_sums)
    A_new.data = A.data * damping + (1 - damping) * A_new.data
    return A_new

In [27]:
def labels(A, R):
    temp = A.copy()
    temp.data += R.data
    ind_data, _, _ = row_max(temp)

    return [mapper[idx] for idx, mapper in zip(ind_data, idx_to_col)]

In [28]:
max_iters = 300
exemplars_log = []

In [29]:
start_time = datetime.now()

In [30]:
for _ in range(max_iters):
    R = update_r(A, S, R)
    A = update_a(R, A)
    c = labels(A, R)

    exemplars_log.append(c)

end_time = datetime.now()

/usr/local/lib/python3.6/dist-packages/scipy/sparse/compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [31]:
timedelta = end_time - start_time

print(f"Training took {timedelta.seconds // 60} minutes")

Training took 26 minutes


Some statistics.

In [32]:
counter = collections.Counter(exemplars_log[-1])

In [33]:
print(f"Number of exemplars (clusters): {len(counter)}")

Number of exemplars (clusters): 29227


In [34]:
counter.most_common(5)

[(2, 602), (1237, 464), (1, 335), (1935, 299), (400, 296)]

### Recommendations

Hide check-ins from the part of users.

In [35]:
df_exemplars = pd.DataFrame({
    'user_id': np.arange(n_users),
    'exemplar_id': exemplars_log[-1]
})

In [36]:
df_exemplars.loc[:, 'user_id'] = df_exemplars['user_id'].apply(map_ids, args=[idx_to_uid]).values
df_exemplars.loc[:, 'exemplar_id'] = df_exemplars['exemplar_id'].apply(map_ids, args=[idx_to_uid]).values

In [37]:
train_perc = 90
n_train = n_users // 100 * train_perc

In [38]:
n_train

89550

In [39]:
users = df_exemplars['user_id'].unique()
np.random.shuffle(users)
train, test = users[:n_train], users[n_train:]

Count check-ins for every cluster.

In [40]:
clusters = collections.defaultdict(list)

for user_id, exemplar_id in df_exemplars.itertuples(index=False): 
    if not (exemplar_id == user_id and exemplar_id in test):
        clusters[exemplar_id].append(user_id)

In [41]:
locations = collections.defaultdict(collections.Counter)

for exemplar_id, cluster in clusters.items():
    checkins = df_checkins[df_checkins['user_id'].isin(cluster)]['location_id'].values
    locations[exemplar_id].update(checkins)

Get recommendations for the test set.

In [42]:
df_test = df_exemplars[df_exemplars['user_id'].isin(test)]

In [43]:
n_top = 10

In [44]:
test_checkins = {}
recommendations = {}

for user_id, exemplar_id in df_test.itertuples(index=False):
    checkins = df_checkins[df_checkins['user_id'] == user_id]['location_id'].values
    test_checkins[user_id] = checkins

    top = [loc_id for loc_id, counts in locations[exemplar_id].most_common(n_top)]
    recommendations[user_id] = top

Calculate the accuracy.

In [45]:
scores = []

for user_id, checkins in test_checkins.items():
    top = recommendations[user_id]
    n_misses = n_top - len(top)

    acc = np.maximum(0, (len(set(top) & set(checkins)) - n_misses) / n_top)
    scores.append(acc)

In [46]:
np.mean(scores)

0.18054529112154202

In [47]:
collections.Counter(scores)

Counter({0.0: 5123,
         0.2: 911,
         0.1: 1533,
         0.7: 178,
         0.3: 562,
         0.5: 273,
         0.4: 420,
         1.0: 480,
         0.6: 235,
         0.9: 152,
         0.8: 146})

In [48]:
n_empty = sum(not val for val in recommendations.values())

print(f"Number of empty recommendations lists: {n_empty}")

Number of empty recommendations lists: 904
